In [75]:
import pandas as pd
import numpy as np
import os

In [111]:
project_dir = "/home/data/dataset/LUAD_asian_8_sample"
sample_sheet = pd.read_csv(os.path.join(project_dir, "GDC Sample.tsv"), sep="\t")

In [ ]:
sample_sheet.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,49c80597-6cd3-4d54-9115-5776d1c44c3a,673621c0-f5d4-4eea-a0ae-469214a70e82.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-78-7166,TCGA-78-7166-01A,Primary Tumor
1,7528d4c5-25d1-4200-90b8-177e3874f2c4,22002e84-1d7a-4f93-9934-9d74f45195c0.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-55-7914,TCGA-55-7914-01A,Primary Tumor
2,22efea62-586e-4e47-acf5-d5839a8ef7fc,96f7dff7-660b-4700-88b3-6479ad02f20c.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-55-1594,TCGA-55-1594-01A,Primary Tumor
3,29e5c4a5-3659-44c1-8609-82af113494e0,f48a9d85-c3c7-40bc-9215-760ba3b8fa60.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-38-4631,TCGA-38-4631-01A,Primary Tumor
4,e8f2cdea-1430-43ad-8359-ead8b4c5fd6e,aea4fa60-a890-453e-8ba6-6f4f3d5ed084.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-73-4670,TCGA-73-4670-01A,Primary Tumor
...,...,...,...,...,...,...,...,...
595,ea4712c3-719a-49d1-9985-1a0f955beec4,731c955b-e1e8-462a-9c94-64310f2d7eae.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-MN-A4N4,TCGA-MN-A4N4-01A,Primary Tumor
596,ba3d2868-fdcf-4c29-9fc6-5c2284ab27eb,981bfd73-1d4c-47c0-870a-b9a6300858dc.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-05-4415,TCGA-05-4415-01A,Primary Tumor
597,4fc71e83-8aab-449f-807a-db5d6eb21ab9,99c5c1c4-851a-42a4-b5a2-dd9da2959343.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-75-5147,TCGA-75-5147-01A,Primary Tumor
598,4699cd8b-a11f-4151-a2ed-0618a476800b,9aa5eb7d-1d54-47f6-aeeb-59be4880f004.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-86-8280,TCGA-86-8280-01A,Primary Tumor


In [113]:
count = 0
pair_case_df = pd.DataFrame()
gb_case_ID = sample_sheet.groupby("Case ID")

for case_ID in gb_case_ID.groups:
    sub_df = gb_case_ID.get_group(case_ID)
    sample_types = sub_df["Sample Type"]
    if ("Primary Tumor" in sample_types.unique()) and ("Solid Tissue Normal" in sample_types.unique()):
        tumor = sub_df.loc[sample_types == "Primary Tumor"].iloc[-1]
        normal = sub_df.loc[sample_types == "Solid Tissue Normal"].iloc[-1]
        pair_case_df = pd.concat([pair_case_df, tumor, normal], axis=1, ignore_index=True)
        count += 1 
        
print(count)
pair_case_df = pair_case_df.T
pair_case_df.to_csv("../all_pair_case_sample_sheet.tsv", index=False, sep="\t")
pair_case_IDs = pair_case_df.loc[:, "Case ID"].unique()
np.random.seed(42)
random_pair_case_IDs = np.random.choice(pair_case_IDs, 
                                        size=10, 
                                        replace=False)
filter = pair_case_df.loc[:, "Case ID"].isin(random_pair_case_IDs)
final_df = pair_case_df[filter]
final_df.loc[:, "Sample Type"] = final_df.loc[:, "Sample Type"].mask(final_df["Sample Type"] == "Primary Tumor", "T")
final_df.loc[:, "Sample Type"] = final_df.loc[:, "Sample Type"].mask(final_df["Sample Type"] == "Solid Tissue Normal", "N")
final_df.loc[:, "New sample ID"] = final_df.loc[:, "Case ID"] + "-" + final_df.loc[:, "Sample Type"]
final_df.to_csv("../10_sample_sheet.tsv", index=False, sep="\t")

58


/tmp/ipykernel_950597/662996149.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[:, "New sample ID"] = final_df.loc[:, "Case ID"] + "-" + final_df.loc[:, "Sample Type"]


In [114]:
final_df

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,New sample ID
0,a35d80d8-8b94-4f66-b408-bbe19f3edd54,9301f86d-9d6f-4aa6-a640-a543b5aea5db.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-38-4625,TCGA-38-4625-01A,T,TCGA-38-4625-T
1,1411e621-7917-4b82-98f2-b56e13f1eb0d,bc46dd0e-3136-4d9b-8fc3-dee56d6e39cf.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-38-4625,TCGA-38-4625-11A,N,TCGA-38-4625-N
10,35a81dbe-bf1e-478c-8a95-d227b4195f34,ab589a7b-7cc9-4257-8fd7-241d2e3658da.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-2657,TCGA-44-2657-01A,T,TCGA-44-2657-T
11,e64627cf-39a9-4da5-af0a-d5f2b527d3e7,1bd869a0-98a7-4e19-994f-41a1ce2c7b2d.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-2657,TCGA-44-2657-11A,N,TCGA-44-2657-N
24,7e6b813e-6bf9-4492-96c3-24f2dd780e53,9e504ab4-85a0-465d-be52-0c28ed83e5f8.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-5645,TCGA-44-5645-01A,T,TCGA-44-5645-T
25,119e2147-2152-41f7-b628-46a5b0d41fb5,6af2f60a-15cf-401e-8e3b-e44637fbd92e.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-5645,TCGA-44-5645-11A,N,TCGA-44-5645-N
26,6f00af19-888a-4b18-bafc-571ce343fae5,df2ac75d-208d-4cc1-867b-aea3e36117c6.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-6145,TCGA-44-6145-01A,T,TCGA-44-6145-T
27,7b4e7bce-6b96-4117-8fb4-4a7952d23941,32ff57d1-3fde-461b-be45-e49365149543.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-6145,TCGA-44-6145-11A,N,TCGA-44-6145-N
50,ab62a7c8-6ed3-4996-836f-580c0b4dc71f,7f9b4506-b00b-4b6f-8469-d0e4a73df1d4.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-49-6745,TCGA-49-6745-01A,T,TCGA-49-6745-T
51,87191941-8fd5-4ec5-9759-4e62f3b2c577,c6bc9e5e-4116-4179-9a22-b75fc6bea73e.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-49-6745,TCGA-49-6745-11A,N,TCGA-49-6745-N


In [110]:
df[df.loc[:, "File Name"].str.contains("genomic")]

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
4,63da5a36-0ec0-4d89-be9d-7319f0eae8ed,f8cc4515-2576-4a72-9cf1-28dfe774eda9.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-55-1595,TCGA-55-1595-01A,Primary Tumor
10,80188ed0-954f-44d9-a212-fe88b8927b62,921aa14f-d674-4ecf-b4fd-ca846acc9fa3.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-55-1596,TCGA-55-1596-01A,Primary Tumor
30,ddede262-9d2b-40b9-ae70-bb4f6c61d2ea,a8d269f5-ebc9-4b2a-9cd9-7019204a246f.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-93-8067,TCGA-93-8067-01A,Primary Tumor
45,3246816d-2008-4930-ad33-8f108ad011d1,98f4637a-4953-4f52-b90a-c768d005a3e7.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-97-8171,TCGA-97-8171-01A,Primary Tumor
47,6d01eab0-89e8-4b6e-a90f-c79f4ca814d2,dd6e2735-7d83-4bc4-87b3-05a5d7f08caa.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-71-8520,TCGA-71-8520-01A,Primary Tumor
202,048cb462-0a42-480a-80da-e5f2035e6abf,094bd5ef-b1ca-46c6-b717-2519e5a6151b.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-71-6725,TCGA-71-6725-01A,Primary Tumor
314,cb267187-4eaa-4670-8a03-ec3c6c8c82b5,b4a1a796-8bce-4b44-b4ed-fd7bf4c81b3b.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-93-A4JP,TCGA-93-A4JP-01A,Primary Tumor
564,a626ad4b-579d-4f16-960e-588f780e809f,91e79d93-01a8-4908-805a-12678d68a543.rna_seq.g...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-86-A456,TCGA-86-A456-01A,Primary Tumor


In [108]:
final_df

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,New sample ID
0,82acdb70-9953-4822-b2b3-fb07fc13e7a4,9301f86d-9d6f-4aa6-a640-a543b5aea5db.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-38-4625,TCGA-38-4625-01A,T,TCGA-38-4625-T
1,3bc6169f-9abb-4899-bd99-f0ee5eb09dd9,bc46dd0e-3136-4d9b-8fc3-dee56d6e39cf.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-38-4625,TCGA-38-4625-11A,N,TCGA-38-4625-N
10,95b28ae9-ad59-4da9-9a64-e0ac4b478377,ab589a7b-7cc9-4257-8fd7-241d2e3658da.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-2657,TCGA-44-2657-01A,T,TCGA-44-2657-T
11,4d27b70e-72f7-4ab1-82fc-cecdef2b9648,1bd869a0-98a7-4e19-994f-41a1ce2c7b2d.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-2657,TCGA-44-2657-11A,N,TCGA-44-2657-N
24,0a44486c-4adf-4040-84b4-7866bb2b7d5b,d6e945f2-4a94-48ab-9ba0-d4f0ee2fb262.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-5645,TCGA-44-5645-01A,T,TCGA-44-5645-T
25,24b036c3-2837-49fc-baa2-1a51e04c4f60,6af2f60a-15cf-401e-8e3b-e44637fbd92e.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-5645,TCGA-44-5645-11A,N,TCGA-44-5645-N
26,4f8ce1cb-18ff-43bf-ba84-4a89ac5f9c86,df2ac75d-208d-4cc1-867b-aea3e36117c6.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-6145,TCGA-44-6145-01A,T,TCGA-44-6145-T
27,d7373740-af07-4fdf-853a-c85524caa229,32ff57d1-3fde-461b-be45-e49365149543.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-44-6145,TCGA-44-6145-11A,N,TCGA-44-6145-N
50,f7c4843f-da83-4265-8aa1-73b55c5ca50b,7f9b4506-b00b-4b6f-8469-d0e4a73df1d4.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-49-6745,TCGA-49-6745-01A,T,TCGA-49-6745-T
51,cff37390-e99d-4acf-9ee8-70e5254f62be,c6bc9e5e-4116-4179-9a22-b75fc6bea73e.rna_seq.c...,Sequencing Reads,Aligned Reads,TCGA-LUAD,TCGA-49-6745,TCGA-49-6745-11A,N,TCGA-49-6745-N
